
# 05 · Vergleich Stichprobe vs. Statista

Dieses Notebook vergleicht die **Stichprobe 2025** mit den harmonisierten **Statista-Jahren** (Standard: 2024).
Es erzeugt nachvollziehbare Abbildungen (Prozentwerte, 0–100 %) und zeigt **Differenzen in Prozentpunkten (pp)**.

Optional: ein globaler Chi²-Goodness-of-Fit-Test (falls `scipy` verfügbar) prüft, ob sich die Verteilung
der Stichprobe insgesamt von Statista unterscheidet.


In [12]:

# 05_vergleich_stichprobe — Cell 1: Setup & Daten laden
from __future__ import annotations

from pathlib import Path
import json
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

BASE = Path.cwd().resolve().parents[0] if Path.cwd().name.lower()=="notebooks" else Path.cwd()
OUT  = BASE / "data" / "processed"
FIG  = BASE / "reports" / "figures"
FIG.mkdir(parents=True, exist_ok=True)

CONFIG = json.loads((OUT / "project_config.json").read_text(encoding="utf-8"))
KANON  = CONFIG["kanon"]

# Daten laden
stat_wide = pd.read_csv(OUT / "statista_harmonisiert_2021_2022_2024.csv").set_index("Kategorie").reindex(KANON).fillna(0.0)
sample_wide = pd.read_csv(OUT / "umfrage_2025_wide.csv").set_index("Kategorie").reindex(KANON).fillna(0.0)

# Spalten in int zahlen (z. B. "2021" -> 2021)
def _to_int_if_year(c):
    s = str(c).strip()
    if s.endswith(".0") and s[:-2].isdigit():
        s = s[:-2]
    return int(s) if s.isdigit() else s
stat_wide.columns   = [_to_int_if_year(c) for c in stat_wide.columns]
sample_wide.columns = [_to_int_if_year(c) for c in sample_wide.columns]

YEARS = sorted([c for c in stat_wide.columns if isinstance(c, int)])
print("Statista Jahre:", YEARS, "| Sample Jahre:", list(sample_wide.columns))

COMPARE_YEAR = 2024  # Zieljahr Statista für den Vergleich
assert COMPARE_YEAR in stat_wide.columns, f"Statista-Zieljahr {COMPARE_YEAR} fehlt."
assert 2025 in sample_wide.columns, "Stichprobenjahr 2025 fehlt im Sample-Wide CSV."

ref = stat_wide[COMPARE_YEAR].rename("Statista")
samp = sample_wide[2025].rename("Stichprobe 2025")

comp = pd.concat([ref, samp], axis=1).fillna(0.0)
comp["Δ pp (Sample - Statista)"] = comp["Stichprobe 2025"] - comp["Statista"]
display(comp.head(10))


Statista Jahre: [2021, 2022, 2024] | Sample Jahre: [2025]


,Statista,Stichprobe 2025,Δ pp (Sample - Statista)
Kategorie,,,
Kleidung / Schuhe,106.0,69.230769,-36.769231
"Elektronik (z. B. Smartphones, Haushaltsgeräte)",38.0,43.589744,5.589744
Lebensmittel / Getränke,16.0,15.384615,-0.615385
Bücher / Medien / Software,25.0,46.153846,21.153846
Medikamente / Drogerieartikel,51.0,28.205128,-22.794872
Hobby- & Freizeitartikel,57.0,53.846154,-3.153846
Möbel / Wohnaccessoires,15.0,0.000000,-15.000000


In [13]:
# Einheitlicher Stil + robustes Layout
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

plt.rcParams.update({
    "figure.dpi": 120,
    "savefig.dpi": 300,
    "font.size": 11,
    "axes.titlesize": 12,
    "axes.labelsize": 11,
    "axes.grid": True,
    "grid.alpha": 0.2,
    "axes.spines.top": False,
    "axes.spines.right": False,
    # Empfehlung: kein autolayout, wir nutzen constrained_layout gezielt pro Figure
    "figure.autolayout": False,
})

def percent_axis(ax, axis="y", decimals=0, limit=(0, 100), pad_pct=0):
    """
    Prozent-Achse 0–100 % mit optionaler rechter/linker Padding-Reserve (pad_pct in %-Punkten),
    damit äußere Labels nicht abgeschnitten werden.
    """
    lo, hi = limit
    hi_padded = hi + float(pad_pct)
    fmt = mtick.PercentFormatter(xmax=100, decimals=decimals)
    if axis == "y":
        ax.set_ylim(lo, hi_padded)
        ax.yaxis.set_major_formatter(fmt)
    else:
        ax.set_xlim(lo, hi_padded)
        ax.xaxis.set_major_formatter(fmt)

def label_bars(ax, decimals=1, rotate=0, inside=False):
    """
    Beschriftet Balken. inside=True platziert Werte IN den Balken (vermeidet Abschneiden).
    """
    fmt = f"%.{decimals}f %%"
    for c in ax.containers:
        if inside:
            ax.bar_label(c, fmt=fmt, padding=2, fontsize=8, rotation=rotate, label_type="center")
        else:
            ax.bar_label(c, fmt=fmt, padding=2, fontsize=8, rotation=rotate)

def label_hbars_right(ax, bars, decimals=1, dx=1.0):
    """
    Beschriftet H-Balken RECHTS außerhalb, mit kleiner Reserve dx (in %-Punkten).
    Setzt clip_on=False, damit nichts „hart“ an der Achse abgeschnitten wird.
    """
    for rect in bars:
        v = rect.get_width()
        ax.text(v + dx, rect.get_y() + rect.get_height()/2,
                f"{v:.{decimals}f} %", va="center", ha="left", fontsize=9, clip_on=False)

def footer(note: str):
    # Fußnote leicht nach unten, wird durch bbox_inches='tight' mitgenommen
    plt.figtext(0.01, -0.04, note, ha="left", va="top", fontsize=9)

def save_fig(path):
    # 'tight' + pad verhindert Abschneiden (Legende, lange Labels)
    plt.savefig(path, bbox_inches="tight", pad_inches=0.25, facecolor="white")
    plt.close()


In [14]:

# 05_vergleich_stichprobe — Cell 3: Balkendiagramm Sample vs. Statista (je Kategorie)
fig, ax = plt.subplots(figsize=(12, 6), layout="constrained")
# KEIN plt.tight_layout() zusätzlich! (beides zusammen kann stören)

comp[["Statista","Stichprobe 2025"]].plot(kind="bar", ax=ax, width=0.8)
label_bars(ax, decimals=1, rotate=90)
ax.set_title(f"Vergleich je Kategorie: Stichprobe 2025 vs. Statista {COMPARE_YEAR}")
ax.set_xlabel("Kategorie")
ax.set_ylabel("Anteil in %")
percent_axis(ax, axis="y", decimals=0, limit=(0, 120))
ax.legend(title="", frameon=False, loc="upper left", bbox_to_anchor=(1.01, 1.0))
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
footer("Hinweis: Prozentwerte. Stichprobe = Anteil Befragter, die die Kategorie auswählten (bei Multi-Choice).")
save_fig(FIG / "05_sample_vs_statista_bars.png")


C:\Users\admin\AppData\Local\Temp\ipykernel_25108\1345936679.py:13: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [15]:

# 05_vergleich_stichprobe — Cell 4: Differenzen (pp)
comp_sorted = comp.sort_values("Δ pp (Sample - Statista)", ascending=False)

fig, ax = plt.subplots(figsize=(12, 6), layout="constrained")
# KEIN plt.tight_layout() zusätzlich! (beides zusammen kann stören)

ax.barh(comp_sorted.index, comp_sorted["Δ pp (Sample - Statista)"].values)
for rect, v in zip(ax.patches, comp_sorted["Δ pp (Sample - Statista)"].values):
    ax.text((0.5 if v>=0 else -0.5) + v, rect.get_y() + rect.get_height()/2,
            f"{v:+.1f} pp", va="center", ha="left" if v>=0 else "right", fontsize=9)
ax.set_title("Differenzen Stichprobe 2025 – Statista 2024 (Prozentpunkte)")
ax.set_xlabel("Δ in Prozentpunkten (pp)")
# symmetrische Skala um 0
left  = min(-10, float(comp_sorted["Δ pp (Sample - Statista)"].min())-2)
right = max( 10, float(comp_sorted["Δ pp (Sample - Statista)"].max())+2)
ax.set_xlim(left, right)
ax.axvline(0, linewidth=1)
plt.tight_layout()
footer("Anmerkung: Δ in Prozentpunkten (pp). Positiv = Stichprobe höher; Negativ = Stichprobe niedriger.")
save_fig(FIG / "05_sample_minus_statista_diffs.png")


C:\Users\admin\AppData\Local\Temp\ipykernel_25108\1751339213.py:18: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


In [16]:

# 05_vergleich_stichprobe — Cell 5: (Optional) globaler Chi²-Test
# Prüft, ob sich die Verteilung (über alle Kategorien) unterscheidet.
# Voraussetzung: Sample zählt Auswahl je Kategorie (cnt) und Summe der Statista-Proportionen ist > 0.

try:
    from scipy.stats import chisquare
    # Sample counts rekonstruieren aus share_% und N (aus 04 exportiert)
    sample_long = pd.read_csv(OUT / "umfrage_2025_long.csv")
    # Respondenten N schätzen (cnt ist pro Kategorie; bei single == N, bei multi Auswahlzählungen)
    # Wir nehmen N = Anzahl Zeilen der Originaldatei nicht direkt; stattdessen:
    # Für single-choice: Summe(cnt) ~ N; für multi-choice: cnt_sum >= N; wir nutzen die max(cnt) als Proxy für N.
    N_proxy = int(sample_long["cnt"].max())
    # Erwartete Counts gemäß Statista (p * N_proxy)
    exp = (stat_wide[COMPARE_YEAR] / 100.0) * N_proxy
    obs = (sample_long.set_index("Kategorie").reindex(KANON)["cnt"]).fillna(0.0)
    # Beide Vektoren angleichen
    exp = exp.reindex(KANON).fillna(0.0)
    obs = obs.reindex(KANON).fillna(0.0)
    # Nur positive Erwartungen verwenden (chi² Voraussetzung)
    mask = exp > 0
    chi2, p = chisquare(f_obs=obs[mask], f_exp=exp[mask])
    print(f"Chi²-Test (global): chi2 = {chi2:.2f}, p = {p:.4f} (N≈{N_proxy})")
except Exception as e:
    print("Chi²-Test übersprungen (scipy nicht verfügbar oder unpassende Daten). Hinweis:", e)


Chi²-Test übersprungen (scipy nicht verfügbar oder unpassende Daten). Hinweis: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1.4901161193847656e-08, but the percent differences are:
0.20250120250120254
